# ml.p4de.24xlarge 인스턴스로 테스트

이 실습은 다음 예시를 참고했습니다. https://github.com/huggingface/trl/blob/main/examples/research_projects/stack_llama/scripts/rl_training.py


In [ ]:
%pip install --disable-pip-version-check -q \
    torch==2.0.1 \
    transformers==4.34.1 \
    datasets==2.12.0 \
    accelerate==0.23.0 \
    evaluate==0.4.0 \
    trl==0.7.2 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    typing_extensions==4.7.1 \
    bitsandbytes==0.41.1 \
    peft==0.5.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%store -r peft_ranking_reward_public_qanda_checkpoint

In [ ]:
print(peft_ranking_reward_public_qanda_checkpoint)

./peft_ranking_reward_public_qanda/


In [ ]:
from dataclasses import dataclass, field
from typing import Optional

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

tqdm.pandas()

In [ ]:
peft_fine_tuned_with_ranking_rewards_llama2_checkpoint = './peft_fine_tuned_with_ranking_rewards_llama2'

@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default="NousResearch/Llama-2-7b-hf", metadata={"help": "the model name"})
    tokenizer_name: Optional[str] = field(default="NousResearch/Llama-2-7b-hf", metadata={"help": "the tokenizer name"})
    reward_model_name: Optional[str] = field(default=peft_ranking_reward_public_qanda_checkpoint, metadata={"help": "the reward model name"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    output_max_length: Optional[int] = field(default=128, metadata={"help": "maximum length for generation"})
    mini_batch_size: Optional[int] = field(default=1, metadata={"help": "the PPO minibatch size"})
    batch_size: Optional[int] = field(default=1, metadata={"help": "the batch size"})
    ppo_epochs: Optional[int] = field(default=4, metadata={"help": "the number of ppo epochs"})
    gradient_accumulation_steps: Optional[int] = field(
        default=1, metadata={"help": "the number of gradient accumulation steps"}
    )
    adafactor: Optional[bool] = field(default=False, metadata={"help": "whether to use the adafactor optimizer"})
    early_stopping: Optional[bool] = field(default=False, metadata={"help": "whether to early stop"})
    target_kl: Optional[float] = field(default=0.1, metadata={"help": "kl target for early stopping"})
    reward_baseline: Optional[float] = field(
        default=0.0,
        metadata={"help": "a baseline value that is subtracted from the reward"},
    )
    batched_gen: Optional[bool] = field(default=False, metadata={"help": "whether to use the batched text gen"})
    save_freq: Optional[int] = field(default=None, metadata={"help": "n steps to save the model"})
#    output_dir: Optional[str] = field(default="runs/", metadata={"help": "n steps to save the model"})
    seed: Optional[int] = field(default=0, metadata={"help": "the seed"})
    steps: Optional[int] = field(default=100, metadata={"help": "number of epochs"})
    init_kl_coef: Optional[float] = field(
        default=0.2,
        metadata={"help": "Initial KL penalty coefficient (used for adaptive and linear control)"},
    )

    adap_kl_ctrl: Optional[bool] = field(default=True, metadata={"help": "Use adaptive KL control, otherwise linear"})

In [ ]:
parser = HfArgumentParser(ScriptArguments)
script_args: ScriptArguments = parser.parse_args_into_dataclasses(return_remaining_strings=True)[0]

dataset_name = "lvwerra/stack-exchange-paired"
config = PPOConfig(
    steps=script_args.steps,
    model_name=script_args.model_name,
    learning_rate=script_args.learning_rate,
    log_with=script_args.log_with,
    batch_size=script_args.batch_size,
    mini_batch_size=script_args.mini_batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    optimize_cuda_cache=True,
    early_stopping=script_args.early_stopping,
    target_kl=script_args.target_kl,
    ppo_epochs=script_args.ppo_epochs,
    seed=script_args.seed,
    init_kl_coef=script_args.init_kl_coef,
    adap_kl_ctrl=script_args.adap_kl_ctrl,
)

# 우리는 감정 분석 파이프라인에 전달할 인자를 정의합니다.
#`return_all_scores`를 True로 설정해 각 토큰의 감정 분류 점수를 얻습니다.
sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16,
    "truncation": True,
}

tokenizer = AutoTokenizer.from_pretrained(script_args.tokenizer_name)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token

In [3]:
# 아래는 데이터 세트를 구축하는 함수 예시입니다.
# 여기서는 datasets 라이브러리의 IMDB 데이터 세트를 사용합니다.
# 이 함수는 자체 데이터 세트에서 모델을 학습시키기 위해 사용자 정의되어야 합니다.

def build_dataset(
    tokenizer,
    dataset_name="lvwerra/stack-exchange-paired",
    data_dir="data/rl",
    split="train"
):
    """
    학습을 위한 데이터 세트를 구축합니다. `load_dataset`에서 데이터 세트를 구축하며, 모델을 자신의 데이터 세트로 학습하려면 이 함수를 맞춤화해야 합니다.

    인자:
        dataset_name (`str`):
            적재할 데이터 세트 이름입니다.

    반환값:
        dataloader (`torch.utils.data.DataLoader`):
            데이터 세트에 대한 데이터 로더입니다.
    """

    ds = load_dataset(dataset_name, data_dir=data_dir, split=split)
    original_columns = ds.column_names
    num_proc = 24

    def preprocess_function(examples):
        new_examples = {
            "query": [],
            "input_ids": [],
        }
        for question in examples["question"]:
            query = "Question: " + question + "\n\nAnswer: "
            tokenized_question = tokenizer(query, truncation=True)
            new_examples["query"].append(query)
            new_examples["input_ids"].append(tokenized_question["input_ids"])

        return new_examples

    ds = ds.map(
        preprocess_function,
        batched=True,
        num_proc=num_proc,
        remove_columns=original_columns,
    )
    ds = ds.filter(lambda x: len(x["input_ids"]) < 512, batched=False)

    ds.set_format(type="torch")
    return ds

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [ ]:
# build_dataset 함수를 호출해 데이터 로더를 가져옵니다.
train_dataset = build_dataset(tokenizer, "lvwerra/stack-exchange-paired", data_dir="data/rl", split="train")

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Found cached dataset parquet (/root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-e5ccc5f74f1da5b7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Map (num_proc=24):   0%|          | 0/7435908 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Filter:   0%|          | 0/7435908 [00:00<?, ? examples/s]

In [ ]:
# value 헤드를 초기화하기 전에 시드를 설정해 결정적인 평가를 보장합니다.
set_seed(config.seed)

# 이제 모델, 참조 모델 및 토크나이저를 구축합니다.
current_device = Accelerator().local_process_index

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
from trl import create_reference_model

model = AutoModelForCausalLMWithValueHead.from_pretrained(
    config.model_name,
    load_in_8bit=True,
    device_map={"": current_device},
    peft_config=lora_config,
)

ref_model = create_reference_model(model)
print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 6746808321
percentage of trainable model parameters: 0.00%



In [ ]:
# 그런 다음 모델, 참조 모델 및 토크나이저를 전달해 PPOTrainer를 구축합니다.
optimizer = None
if script_args.adafactor:
    optimizer = Adafactor(
        filter(lambda p: p.requires_grad, model.parameters()),
        scale_parameter=False,
        relative_step=False,
        warmup_init=False,
        lr=config.learning_rate,
    )

ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=train_dataset,
    data_collator=collator,
    optimizer=optimizer,
)


# 보상 모델을 사용해 감정 분석 파이프라인을 구축합니다.
# 모델 이름과 감정 분석 파이프라인 인자를 전달합니다.
# 또한 PPOTrainer와 동일한 장치에 설정되도록 합니다.
reward_model_tokenizer = AutoTokenizer.from_pretrained(script_args.reward_model_name)

device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a ` pipeline` bug
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model=script_args.reward_model_name,
    device_map={"": current_device},
    model_kwargs={"load_in_8bit": True},
    tokenizer=reward_model_tokenizer,
    return_token_type_ids=False,
)

# `generate` 함수에 전달할 인자를 정의합니다.
# 이 인자는 PPOTrainer의 `generate` 함수에 전달되며, 이는 학습된 모델의 `generate` 함수에 대한 래퍼입니다.
generation_kwargs = {
    # "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": 100_000,
}
output_min_length = 32
output_max_length = script_args.output_max_length
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [ ]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if epoch >= config.total_ppo_epochs:
        break

    question_tensors = batch["input_ids"]

    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        length_sampler=output_length_sampler,
        **generation_kwargs,
    )
    batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

    # 감정 분석 파이프라인을 사용해 보상 점수를 계산합니다.
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[0]["score"] - script_args.reward_baseline) for output in pipe_outputs]

    # PPO 단계 수행
    stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

    if script_args.save_freq and epoch and epoch % script_args.save_freq == 0:
        #ppo_trainer.save_pretrained(script_args.output_dir + f"step_{epoch}")
        ppo_trainer.tokenizer.save_pretrained(peft_fine_tuned_with_ranking_rewards_llama2_checkpoint)
        ppo_trainer.accelerator.unwrap_model(ppo_trainer.model).save_pretrained(peft_fine_tuned_with_ranking_rewards_llama2_checkpoint) # merge
        #ppo_trainer.model.save_pretrained(peft_fine_tuned_with_detoxification_rewards_checkpoint)

0it [00:00, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/si

In [ ]:
ppo_trainer.tokenizer.save_pretrained(peft_fine_tuned_with_ranking_rewards_llama2_checkpoint)
ppo_trainer.accelerator.unwrap_model(ppo_trainer.model).save_pretrained(peft_fine_tuned_with_ranking_rewards_llama2_checkpoint) # merge?

# 평가

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, AutoPeftModelForCausalLM

import torch
import evaluate

In [ ]:
ppo_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_fine_tuned_with_ranking_rewards_llama2_checkpoint,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ref_model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(
    toxicity_model_name,
    device_map="auto"
)
toxicity_model = AutoModelForSequenceClassification.from_pretrained(
    toxicity_model_name,
    device_map="auto"
)
print(toxicity_model.config.id2label)

{0: 'nothate', 1: 'hate'}


유해하지 않는 텍스트를 가져와서 토큰화하고 모델에게 전달합니다. 출력 로짓값, 확률 및 미세 조정을 위해 사용할 보상 점수를 출력합니다.



In [ ]:
non_toxic_text = "You are a great person and i like you."

toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(input_ids=toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# [혐오 아님, 혐오]에 대한 확률 출력
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# '혐오 아님'에 대한 로짓값을 가져옵니다 - 이것이 보상입니다!
not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (value of "not hate" logit): {nothate_reward}')

logits [not hate, hate]: [4.6532111167907715, -4.178227424621582]
probabilities [not hate, hate]: [0.9998539686203003, 0.0001460467028664425]
reward (value of "not hate" logit): [4.6532111167907715]


유해한 댓글을 보여주겠습니다. 이는 더 유해하므로 보상이 낮을 것입니다.


In [ ]:
toxic_text = "You are a terrible person and i hate you."

toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# [혐오 아님, 혐오]에 대한 확률 출력
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# '혐오 아님'에 대한 로짓값을 가져옵니다 - 이것이 보상입니다!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (value of "not hate" logit): {nothate_reward}')

logits [not hate, hate]: [-2.064443349838257, 1.6650441884994507]
probabilities [not hate, hate]: [0.023442398756742477, 0.9765575528144836]
reward (value of "not hate" logit): [-2.064443349838257]


유해성 보상 모델을 위한 코드를 단순화하기 위해 허깅 페이스 추론 파이프라인을 설정하세요.

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

sentiment_pipe = pipeline("sentiment-analysis",
                          model=toxicity_model_name,
                          device=device)
reward_logits_kwargs = {
    "top_k": None, # 모든 점수를 반환합니다.
    "function_to_apply": "none", # "none"으로 설정해 원시 로짓값을 검색합니다.
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None, # 모든 점수를 반환합니다.
    "function_to_apply": "softmax", # "softmax"로 설정해 소프트맥스를 적용하고 확률을 검색합니다.
    "batch_size": 16
}

print("Reward model output for non-toxic text:")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))
print("\nReward model output for toxic text:")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Reward model output for non-toxic text:
[{'label': 'nothate', 'score': 4.6532111167907715}, {'label': 'hate', 'score': -4.178227424621582}]
[{'label': 'nothate', 'score': 0.9998539686203003}, {'label': 'hate', 'score': 0.0001460467028664425}]

Reward model output for toxic text:
[{'label': 'hate', 'score': 1.6650441884994507}, {'label': 'nothate', 'score': -2.064443349838257}]
[{'label': 'hate', 'score': 0.9765575528144836}, {'label': 'nothate', 'score': 0.023442398756742477}]


출력은 `nothate`(긍정) 클래스와 `hate`(부정) 클래스 모두에 대한 로짓값입니다. 하지만 PPO는 LLM 출력의 유해성을 줄이는 데 도움이 되는 긍정적인 보상 신호로 `nothate` 클래스의 로짓값만 사용할 것입니다.

In [ ]:
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))

[{'label': 'nothate', 'score': 4.6532111167907715}, {'label': 'hate', 'score': -4.178227424621582}]
[{'label': 'nothate', 'score': 0.9998539686203003}, {'label': 'hate', 'score': 0.0001460467028664425}]


In [ ]:
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

[{'label': 'hate', 'score': 1.6650441884994507}, {'label': 'nothate', 'score': -2.064443349838257}]
[{'label': 'hate', 'score': 0.9765575528144836}, {'label': 'nothate', 'score': 0.023442398756742477}]


<a name='2.3'></a>
### 2.3. - 유해성 평가

미세 조정/유해성 제거 전후의 모델을 평가하려면 [유해성 평가 지표](https://huggingface.co/spaces/evaluate-measurement/toxicity)를 설정해야 합니다. **유해성 점수**는 0에서 1 사이의 소수점 값으로, 1이 가장 높은 유해성을 나타냅니다.

In [ ]:
toxicity_evaluator = evaluate.load("toxicity",
                                    toxicity_model_name,
                                    module_type="measurement",
                                    toxic_label="hate")

[2.2](#2.2) 섹션에서 사용한 문장들에 대한 유해성을 계산해 보십시오. 유해성 점수는 보상 모델에서 직접 반환된 `혐오` 클래스의 확률과 같습니다.


In [ ]:
toxicity_score = toxicity_evaluator.compute(predictions=[
    non_toxic_text
])

print("Toxicity score for non-toxic text:")
print(toxicity_score["toxicity"])

toxicity_score = toxicity_evaluator.compute(predictions=[
    toxic_text
])

print("\nToxicity score for toxic text:")
print(toxicity_score["toxicity"])

Toxicity score for non-toxic text:
[0.00014604683383367956]

Toxicity score for toxic text:
[0.9765576720237732]


이 평가기는 [2.1](#2.1) 섹션에서 준비한 대화의 유해성을 계산하는 데 사용될 수 있습니다. 테스트 데이터 세트(`dataset["test"]`), 해당 섹션에서 사용한 것과 동일한 토크나이저, [2.2](#2.2) 섹션에서 준비한 고정된 PEFT 모델, 그리고 유해성 평가기를 전달해야 합니다. 필요한 단계를 `evaluate_toxicity` 함수로 감싸는 것이 편리합니다.

In [ ]:
test_dataset = build_dataset(tokenizer, "lvwerra/stack-exchange-paired", data_dir="data/evaluation", split="train")
test_dataset = test_dataset.select(range(1000))

Map (num_proc=24):   0%|          | 0/4483004 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Filter:   0%|          | 0/4483004 [00:00<?, ? examples/s]

In [ ]:
import numpy as np

def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids.to(device)

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             tok_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # 평균 및 표준 편차는 np를 사용해 계산.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

이제 미세 조정/유해성 제거 전에 모델 유해성을 계산해 보겠습니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(script_args.model_name, device_map="auto")

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=test_dataset,
                                                                          num_samples=100)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

101it [07:13,  4.29s/it]

toxicity [mean, std] before detox: [0.01928018110269441, 0.01788940742928178]


In [ ]:
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=test_dataset,
                                                                        num_samples=100)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

101it [08:40,  5.16s/it]

toxicity [mean, std] after detox: [0.018666810890899437, 0.019985045280750742]



그리고 참조 모델(유해성 제거 전)과 미세 조정된 모델(유해성 제거 이후)의 유해성 점수를 비교합니다.

In [ ]:
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: 3.18%
std: -11.71%


<a name='3.4'></a>
### 3.4 - 모델의 정성적 평가

테스트 데이터 세트에서 몇 가지 예를 살펴보겠습니다. 유해성 평가기를 사용해 원래의 `ref_model`과 미세 조정/유해성이 제거된 `ppo_model`을 비교할 수 있습니다.


​

In [ ]:
batch_size = 100
compare_results = {}

df_batch = test_dataset[0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# ppo 및 기본 모델에서 응답을 가져옵니다.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# 응답을 디코딩.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# 쿼리/응답 쌍의 감정 분석을 유해성 제거 이전과 이후로 수행합니다.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **sent_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **sent_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

100%|██████████| 100/100 [12:12<00:00,  7.32s/it]


결과를 데이터 프레임(DataFrame)으로 저장하고 검토합니다.

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 1000)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Question: I was studying for icing and a tailplane stall. I have looked up some internet pages and instrument flying handbook, and found the procedure below.\n\n1. raise flaps to the previous setting. (To reduce down wash from the main wing so that reducing negative angle of attack of the tail and break the stall)\n2. apply nose up elevator pressure (I don't get it. The nose up pressure will make the elevator to go up and wouldn't this increase the negative angle of attack and worsen the stall?)\n3. do not increase airspeed unless it is necessary to avoid a wing stall. (Why shouldn't we increase airspeed?)\n\nSo now I'm trying to understand the reason why should a pilot do such actions. Can you help me out?\n\nAnswer:","1) I don't know what it is, but assume it is wrong. There is no verifiable source given, so the world's experts are looking at you.\n\n2)","605.3. Pilot Actions at Stall Entry\n\n(a) When the aircraft has a natural or induced stall, the pilot applies up elevator, neutralize ailer",2.287549,3.497573,1.210024
1,"Question: In a C# enumeration, are there any negative side effects of using a negative number?\n\nI am modelling response codes and one of the codes in negative. This compiles but I want to know if there are any negative side effects to this.\n\n```\npublic enum ResponseCodes\n{\n InvalidServerUserPasswordCombo = -1,\n\n // etc.\n}\n\n```\n\nAnswer:","1) Positive number is always useful.\nPositive numbers increase, negative numbers decrease.\n\n2) Negative number will normally not directly used in the application, because it will bring a small trouble for the people who use the number.\n\n3) If you use it, you must also declare it.\n\n\begin{code","0 would be a more appropriate value, but AFAIK, there's no difference between any negative-numbered enumeration.</s><s> Tags: c#, .net, caching, ninject\n\nQuestion: Ninject .NET in C# .NET cache dependencies\n\nI would like to store some data in cache (the data are fetch",2.192516,3.255872,1.063356
2,"Question: Isn't that nicely recursive? \n\nI've got a portable command prompt on my external drive, and it has a nice .bat file to configure some initial settings, but I'd like more! \n\nHere's what I know how to set from .bat:\n\n* Colors = (color XY) where x and y are hex digits for the predefined colors\n* Prompt = (prompt $p$g) sets the prompt to ""C:\etc\etc >"" the default prompt\n* Title = (title ""text"") sets the window title to ""text""\n* Screen Size = (mode con: cols=XX lines=YY) sets the columns and lines size of the window\n* Path = (SET PATH=%~d0\bin;%PATH%) sets up local path to my tools and appends the computer's path\n\nSo that's all great. But there are a few settings I can't seem to set from the bat. Like, how would I set these up wihtout using the Properties dialogue:\n\n* Buffer = not screen size, but the buffer\n* Options like quick edit mode and autocomplete\n* Popup colors\n* Font. And can you use a font on the portable drive, or must it be installed to work?\n* ...","6000A.COM is a notable fixed-document JScript notepad,[2] started by Jamie Zawinski in 1997. It is a rewrite of some bits of the original and continues 6000B Джон Сисьорс портал. 6000A.COM is notable because it replaced Notepad in Windows as the default platform documentation\n\nComment: Oh aprendí esto en la Hamilton High School en Phoenix!\n\nComment: To be fair, John Siassor's hoax was pretty good.","1. OS environment variables, such as path, color, prompt, title and window size can be set as normally as we did in Windows commandline.\n2. Options can be set by command prompt by `set` command, but it will are run command prompt loader, so you must gained administrator privileges . For example, `set transient` will set Windows command prompt(default) \ Transient mode.\n\n3. Font can be set by prompt too, but the limitation that we can not simploy change the output like colorful prompt etc. More 